# Client Test

In [21]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'micropython')))
 
import client
from collections import OrderedDict

### Start client

In [22]:
the_client = client.Client()
the_client.start()

My name is Client_366

[connected: ('192.168.0.105', 9662)]
Sending 305 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-01-29 21:57:00.547700'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2017-01-29 21:57:00.547700'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

[Listen to messages]


### Prepare messages

In [23]:
messages = OrderedDict()

messages['read_GPIOs'] = {'type': 'command',
                          'command': 'read GPIOs',
                          'need_result': True}

messages['blink_led'] = {'type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 10, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}} 

In [24]:
# messages['write_GPIOs'] = {'type': 'command',
                           # 'command': 'write GPIOs',
                           # 'kwargs': {'pins_and_values': [(2, 0), (2, 1), (2, 0),]}} 

# messages['test eval'] = {'type': 'eval',
                         # 'to_evaluate': '2+3',
                         # 'need_result': True}                                   

# messages['test exec'] = {'type': 'exec',
                         # 'to_exec': 'print("Testing exec !")'}

# with open('script_to_deploy.py') as f:
    # script = f.read()        
# messages['test upload script'] = {'type': 'script', 
                                  # 'script': script} 


Data received: 400 bytes
Message:
OrderedDict([('correlation_id', '2017-01-29 21:57:00.547700'), ('message_id', '2017-01-29 21:57:00.639700'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_1dsc000': "('192.168.0.104', 18507)", 'NodeMCU_f1d30800': "('192.168.0.103', 5703)", 'Client_366': "('192.168.0.105', 51426)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-29 21:57:00.639700'), ('type', 'result')])



### Ask Hub for a list of connected nodes

In [25]:
while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

In [26]:
message = {'type': 'command',
                   'command': 'list connections by name',
                   'need_result': True}     

_, asynch_result = the_client.request('Hub', message) 

try:
    remote_nodes = sorted(list(asynch_result.get().keys()))

    print ('\n[____________ Connected nodes ____________]\n')        
    print('\nConnected nodes:\n{}\n'.format(remote_nodes)) 
    
except Exception as e:
    print(e)

Sending 276 bytes
Message:
OrderedDict([('command', 'list connections by name'), ('correlation_id', '2017-01-29 21:57:01.066700'), ('message_id', '2017-01-29 21:57:01.066700'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])


Data received: 400 bytes
Message:
OrderedDict([('correlation_id', '2017-01-29 21:57:01.066700'), ('message_id', '2017-01-29 21:57:01.149700'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_1dsc000': "('192.168.0.104', 18507)", 'NodeMCU_f1d30800': "('192.168.0.103', 5703)", 'Client_366': "('192.168.0.105', 51426)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-29 21:57:01.149700'), ('type', 'result')])


[____________ Connected nodes ____________]


Connected nodes:
['Client_366', 'NodeMCU_1dsc000', 'NodeMCU_f1d30800']



### Blink each node

In [27]:
# remote_node = 'NodeMCU_|edca00'
# the_client.request(remote_node, messages['blink_led']) 

# for remote_node in remote_nodes:
#     the_client.request(remote_node, messages['blink_led']) 

### Send out messages and get asynchonous results

In [28]:
print ('\n[______________ Sending messages ______________]\n')
                
results = []

## send out the messages
for message in messages.values():
    for remote_node in remote_nodes:
        if remote_node != the_client.node.worker.name:  # exclude client self
            formatted_message, asynch_result = the_client.request(remote_node, message) 
            results.append((formatted_message, asynch_result))


[______________ Sending messages ______________]

Sending 274 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-29 21:57:01.378700'), ('message_id', '2017-01-29 21:57:01.378700'), ('need_result', True), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

Sending 275 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-29 21:57:01.413700'), ('message_id', '2017-01-29 21:57:01.413700'), ('need_result', True), ('receiver', 'NodeMCU_f1d30800'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

Sending 334 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-01-29 21:57:01.457700'), ('kwargs', {'times': 10, 'on_seconds': 0.1, 'forever': False, 'off_seconds': 0.1}), ('message_id', '2017-01-29 21:57:01.457700'), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])



### Actually get the results

In [29]:
# collect and print results        
print('\n[_________ Wait few seconds for reply _________]\n')

for (message, result) in results:
    try:
        if message.get('need_result'):
            print('\n[Result for request]:\n___Request___:\n{0}\n___Result____:\n{1}\n'.format(message, result.get() if result else None))
    except Exception as e:
        print('\n[{}]\nMessage:\n{}'.format(e, message))


[_________ Wait few seconds for reply _________]


Data received: 320 bytes
Message:
OrderedDict([('correlation_id', '2017-01-29 21:57:01.378700'), ('message_id', '46340'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_1dsc000'), ('result', [[5, 1], [12, 1], [13, 1], [14, 1], [15, 0], [16, 0]]), ('sender', 'NodeMCU_1dsc000'), ('time_stamp', '2017-01-29 21:57:01.667700'), ('type', 'result')])


[Result for request]:
___Request___:
{'sender': 'Client_366', 'message_id': '2017-01-29 21:57:01.378700', 'correlation_id': '2017-01-29 21:57:01.378700', 'need_result': True, 'reply_to': 'Client_366', 'command': 'read GPIOs', 'type': 'command', 'receiver': 'NodeMCU_1dsc000'}
___Result____:
[[5, 1], [12, 1], [13, 1], [14, 1], [15, 0], [16, 0]]


Data received: 323 bytes
Message:
OrderedDict([('correlation_id', '2017-01-29 21:57:01.413700'), ('message_id', '225576'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_f1d30800'), ('result', [[5, 0], [12, 1], [13, 1], [14, 1], [15, 0], [16, 1]]),

### Stop the demo

In [30]:
# Stopping
the_client.stop()
the_client = None
print ('\n[________________ Demo stopped ________________]\n')


[________________ Demo stopped ________________]

